In [1]:
from datasets import load_dataset

# load from parquet file (~4000 samples in a parquet file)
# link to other parquet files: https://huggingface.co/datasets/linhtran92/viet_bud500/tree/main/data

train_url = "https://huggingface.co/datasets/linhtran92/viet_bud500/resolve/main/data/train-00000-of-00105-be5f872f8be772f5.parquet"
test_url = "https://huggingface.co/datasets/linhtran92/viet_bud500/resolve/main/data/test-00000-of-00002-531c1d81edb57297.parquet"

data_files = {"train": train_url, "test" : test_url}
dataset = load_dataset("parquet", data_files=data_files, )

# # load dataset via streaming
# dataset = load_dataset("linhtran92/viet_bud500", split='test', streaming=True)
# dataset.take(2)

# # load all (649158 samples, ~100gb, ~2hrs to complete loading)
# dataset = load_dataset("linhtran92/viet_bud500", split="test")

In [10]:
import soundfile as sf

In [9]:
count = 0
for audio in dataset['train']['audio']:
    sf.write(f'../dataset/audio/train_24k/train24k_{str(count).zfill(10)}.wav', audio['array'], 24000)
    count += 1
for audio in dataset['test']['audio']:
    sf.write(f'../dataset/audio/test_24k/test24k_{str(count).zfill(10)}.wav', audio['array'], 24000)
    count += 1

In [11]:
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='vi', preserve_punctuation=True,  with_stress=True)

In [13]:
from phonemizer import phonemize
text = "Xin chào, tôi là một lập trình viên."

# Sử dụng phonemizer để chuyển đổi văn bản thành IPA
ipa_text = phonemize(text, language='vi', backend='espeak', strip=True, preserve_punctuation=True)

print(ipa_text)

sin tʃaː2w, t̪oj laː2 mo6t̪ lə6p tʃi2ɲ viɛn.


In [12]:
with open('../dataset/train_list_24k.txt', 'w') as f:
    count = 0
    for text in dataset['train']['transcription']:
        text = text.strip()
        ps = global_phonemizer.phonemize([text])
        f.write(f'train_24k/train24k_{str(count).zfill(10)}.wav|{ps[0]}|0\n')
        count += 1
with open('../dataset/test_list_24k.txt', 'w') as f:
    count = 0
    for text in dataset['test']['transcription']:
        text = text.strip()
        ps = global_phonemizer.phonemize([text])
        f.write(f'test_24k/test24k_{str(count).zfill(10)}.wav|{ps[0]}|0\n')
        count += 1